In [1]:
!nvidia-smi

Tue Aug  3 11:11:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import cv2
import PIL
from PIL import Image
import random
import imgaug
from imgaug import augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import time

In [4]:
import torch.nn as nn
import torch
from torchsummary import summary
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torchvision import transforms

In [5]:
sys.path.insert(0, '/content/drive/My Drive/Iñaki.Martínez/Code')

from dataset_handler import DataSet
from segmentation_metrics import get_evaluation_metrics

In [6]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
imgaug.seed(RANDOM_SEED)

In [7]:
N_EPOCHS = 450
INITIAL_EPOCH = 1
GENERATOR_LEARNING_RATE = 0.00005
DISCRIMINATOR_LEARNING_RATE = 0.00005

In [8]:
ngpu = 1
DEVICE = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(f"Pytorch device: {DEVICE}")

Pytorch device: cuda:0


In [9]:
WEIGHTS_PATH = "/content/drive/MyDrive/Iñaki.Martínez/Code/weights/rda-unet-wgan-weights"
PRETRAINED_WEIGHTS = False

In [10]:
N_CRITIC = 5

In [11]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/Iñaki.Martínez/Datasets/Dataset of Breast Ultrasound Images (Aldhyabani et al.)"
TRAIN_DATA_ANNOTATIONS_FILE = "gan_train_bus_images.csv"
VAL_DATA_FILE = "/content/drive/MyDrive/Iñaki.Martínez/Datasets/Dataset of Breast Ultrasound Images (Aldhyabani et al.)"
VAL_DATA_ANNOTATIONS_FILE = "gan_val_bus_images.csv"

BATCH_SIZE = 64

DATA_LOADERS_CORE_NUMBER = 2

In [12]:
def load_img_transforms():

    """
    Funcion que carga las transformaciones

    :return:
    """
    train_data_transform = transforms.Compose([
        transforms.Resize((128, 128), interpolation=PIL.Image.NEAREST),
        transforms.ToTensor()
    ])

    val_data_transform = train_data_transform

    return train_data_transform, val_data_transform

In [13]:
def weights_init(m):

    """
    Inicializacion de los pesos de la red

    :param m: red
    :return:
    """
    classname = m.__class__.__name__

    if classname == 'DilationConvolutionsChain':
        pass
    else:
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)


def merge_images_with_masks(images, masks):

    """
    Genera las imagenes de 4 canales que se pasan al discriminador (3 de la imagen original + 1 con
    la mascara de segmentacion

    :param images: imagenes
    :param masks: mascaras de segmentacion
    :return: tensor con imagenes de 4 canales
    """

    batch_size = images.shape[0]
    img_dim = images.shape[2]
    merged = torch.rand(batch_size, 4, img_dim, img_dim)

    for i in range(batch_size):
        merged[i] = torch.cat((images[i], masks[i]))

    return merged


def gradient_penalty(critic, real_data, fake_data, penalty, device):

    n_elements = real_data.nelement()
    batch_size = real_data.size()[0]
    colors = real_data.size()[1]
    image_width = real_data.size()[2]
    image_height = real_data.size()[3]
    alpha = torch.rand(batch_size, 1).expand(batch_size, int(n_elements / batch_size)).contiguous()
    alpha = alpha.view(batch_size, colors, image_width, image_height).to(device)

    fake_data = fake_data.view(batch_size, colors, image_width, image_height)
    interpolates = alpha * real_data.detach() + ((1 - alpha) * fake_data.detach())

    interpolates = interpolates.to(device)
    interpolates.requires_grad_(True)
    critic_interpolates = critic(interpolates)

    gradients = torch.autograd.grad(
        outputs=critic_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones(critic_interpolates.size()).to(device),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * penalty

    return gradient_penalty

In [14]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

class SegmentationEvaluationMetrics:

    def __init__(self, CCR, precision, recall, sensibility, specifity, f1_score,
                 jaccard, dice, roc_auc, precision_recall_auc, hausdorf_error):
        self.CCR = CCR
        self.precision = precision
        self.recall = recall
        self.sensibility = sensibility
        self.specifity = specifity
        self.f1_score = f1_score
        self.jaccard = jaccard
        self.dice = dice
        self.roc_auc = roc_auc
        self.precision_recall_auc = precision_recall_auc
        self.hausdorf_error = hausdorf_error


def compute_jaccard_dice_coeffs(mask1, mask2):
    """Calculates the dice coefficient for the images"""

    mask1 = np.asarray(mask1).astype(np.bool)
    mask2 = np.asarray(mask2).astype(np.bool)

    if mask1.shape != mask2.shape:
        raise ValueError("Shape mismatch: mask1 and mask2 must have the same shape.")

    mask1 = mask1 > 0.5
    mask2 = mask2 > 0.5

    im_sum = mask1.sum() + mask2.sum()

    if im_sum == 0:
        return 1.0

    # Compute Dice coefficient
    intersection = np.logical_and(mask1, mask2).sum()
    union = im_sum - intersection

    return intersection / union, 2. * intersection / im_sum

def compute_jaccard_coeff(mask1, im2):
    """Calculates the jaccard coefficient for the images"""

    mask1 = np.asarray(mask1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if mask1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    mask1 = mask1 > 0.5
    im2 = im2 > 0.5

    im_sum = mask1.sum() + im2.sum()
    if im_sum == 0:
        return 1.0

    # Compute Dice coefficient
    intersection = np.logical_and(mask1, im2).sum()
    union = im_sum - intersection
    return intersection / union


def get_conf_mat(prediction, groundtruth):
    """Computes scores:
    FP = False Positives
    FN = False Negatives
    TP = True Positives
    TN = True Negatives
    return: FP, FN, TP, TN"""

    prediction = prediction > 0.5
    groundtruth = groundtruth > 0.5

    FP = np.float(np.sum((prediction == 1) & (groundtruth == 0)))
    FN = np.float(np.sum((prediction == 0) & (groundtruth == 1)))
    TP = np.float(np.sum((prediction == 1) & (groundtruth == 1)))
    TN = np.float(np.sum((prediction == 0) & (groundtruth == 0)))


    return FP, FN, TP, TN


def merge_images_with_masks(images, masks):

    """
    Genera las imagenes de 4 canales que se pasan al discriminador (3 de la imagen original + 1 con
    la mascara de segmentacion

    :param images: imagenes
    :param masks: mascaras de segmentacion
    :return: tensor con imagenes de 4 canales
    """

    batch_size = images.shape[0]
    img_dim = images.shape[2]
    merged = torch.rand(batch_size, 4, img_dim, img_dim)

    for i in range(batch_size):
        merged[i] = torch.cat((images[i], masks[i]))

    return merged


def get_evaluation_metrics2(epoch, dataloader, segmentor, DEVICE, writer=None, SAVE_SEGS=False, COLOR=True,
                           N_EPOCHS_SAVE=10, folder=""):

    save_folder = os.path.join(folder, f"epoch_{epoch}")

    if SAVE_SEGS and epoch % N_EPOCHS_SAVE == 0:
      if not os.path.isdir(save_folder):
          os.mkdir(save_folder)

    ccrs = []

    precisions = []
    recalls = []

    sensibilities = []
    specifities = []

    f1_scores = []

    jaccard_coefs = []
    dice_coeffs = []

    roc_auc_coeffs = []
    precision_recall_auc_coeffs = []

    hausdorf_errors = []

    segmentor.eval()

    with torch.no_grad():

        for i, batched_sample in enumerate(dataloader):

          images, masks, filenames = batched_sample["image"].to(DEVICE), batched_sample["mask"].to(DEVICE), \
                                     batched_sample["filename"]

          segmentations = segmentor(images)
          segmentations = torch.autograd.Variable((segmentations > 0.5).float())
          
          trans = transforms.ToPILImage()

          for j in range(images.shape[0]):
              image, mask = images[j].to("cpu"), masks[j].to("cpu")
              segmentation = segmentations[j].to("cpu")
              name = filenames[j].split('/')[-1]

              FP, FN, TP, TN = get_conf_mat(segmentation.numpy(), mask.numpy())

              # print(f"FP = {FP}, FN = {FN}, TP = {TP}, TN = {TN}, TOTAL = {FP + FN + TP + TN}")

              ccr = np.divide(TP + TN, FP + FN + TP + TN)

              precision = np.divide(TP, TP + FP)
              recall = np.divide(TP, TP + FN)

              sensibility = np.divide(TP, TP + FN)
              specifity = np.divide(TN, TN + FP)

              f1_score = 2 * np.divide(precision * recall, precision + recall)

              jaccard_coef, dice_coeff = compute_jaccard_dice_coeffs(segmentation.numpy(), mask.numpy())

              mask_labels = mask.numpy().ravel().astype(np.int32)
              segmentation_labels = segmentation.numpy().ravel()
              fpr, tpr, _ = roc_curve(mask_labels, segmentation_labels)
              roc_auc = auc(fpr,tpr)

              precision_values, recall_values, _ = precision_recall_curve(mask_labels, segmentation_labels)
              precision_recall_auc = auc(recall_values, precision_values)


              hausdorf_error = 12.0

              ccrs.append(ccr)
              precisions.append(precision)
              recalls.append(recall)
              sensibilities.append(sensibility)
              specifities.append(specifity)
              f1_scores.append(f1_score)
              jaccard_coefs.append(jaccard_coef)
              dice_coeffs.append(dice_coeff)
              roc_auc_coeffs.append(roc_auc)
              precision_recall_auc_coeffs.append(precision_recall_auc)
              hausdorf_errors.append(hausdorf_error)

              if SAVE_SEGS and epoch % N_EPOCHS_SAVE == 0:

                  image_save = trans(image)
                  mask_save = trans(mask)
                  segmentation_save = trans(segmentation)

                  opencv_image = np.array(image_save)
                  opencv_image = opencv_image[:, :, ::-1].copy()
                  opencv_gt = np.array(mask_save)
                  opencv_segmentation = np.array(segmentation_save)
                  opencv_segmentation = np.where(opencv_segmentation>0.5, 1.0, 0.0)

                  if not COLOR:
                      img = np.vstack(
                          (cv2.cvtColor(opencv_image, cv2.COLOR_RGB2GRAY), opencv_gt, opencv_segmentation))
                      cv2.imwrite(os.path.join(save_folder, f"{name}.png"), img)

                  else:
                      contours_gt, hierarchy = cv2.findContours(opencv_gt, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                      contours_seg, hierarchy = cv2.findContours(opencv_segmentation, cv2.RETR_TREE,
                                                                  cv2.CHAIN_APPROX_SIMPLE)

                      cv2.drawContours(opencv_image, contours_gt, -1, (0, 255, 0), 1)
                      cv2.drawContours(opencv_image, contours_seg, -1, (0, 0, 255), 1)

                      cv2.imwrite(os.path.join(save_folder, f"{name}.png"), opencv_image)


        ccrs = np.array(ccrs)[~np.isnan(np.array(ccrs))]
        precisions = np.array(precisions)[~np.isnan(np.array(precisions))]
        recalls = np.array(recalls)[~np.isnan(np.array(recalls))]
        sensibilities = np.array(sensibilities)[~np.isnan(np.array(sensibilities))]
        specifities = np.array(specifities)[~np.isnan(np.array(specifities))]
        f1_scores = np.array(f1_scores)[~np.isnan(np.array(f1_scores))]
        jaccard_coefs = np.array(jaccard_coefs)[~np.isnan(np.array(jaccard_coefs))]
        dice_coeffs = np.array(dice_coeffs)[~np.isnan(np.array(dice_coeffs))]
        roc_auc_coeffs = np.array(roc_auc_coeffs)[~np.isnan(np.array(roc_auc_coeffs))]
        precision_recall_auc_coeffs = np.array(precision_recall_auc_coeffs)[~np.isnan(np.array(precision_recall_auc_coeffs))]
        hausdorf_errors = np.array(hausdorf_errors)[~np.isnan(np.array(hausdorf_errors))]

        mean_ccr = np.nansum(ccrs) / len(ccrs)
        std_ccr = np.std(np.array(ccrs))

        mean_precision = np.nansum(precisions) / len(precisions)
        std_precision = np.std(np.array(precisions))

        mean_recall = np.nansum(recalls) / len(recalls)
        std_recall = np.std(np.array(recalls))

        mean_sensibility = np.nansum(sensibilities) / len(sensibilities)
        std_sensibility = np.std(np.array(sensibilities))

        mean_specifity = np.nansum(specifities) / len(specifities)
        std_specifity = np.std(np.array(specifities))

        mean_f1_score = np.nansum(f1_scores) / len(f1_scores)
        std_f1_score = np.std(np.array(f1_scores))

        mean_jaccard_coef = np.nansum(jaccard_coefs) / len(jaccard_coefs)
        std_jaccard_coef = np.std(np.array(jaccard_coefs))

        mean_dice_coeff = np.nansum(dice_coeffs) / len(dice_coeffs)
        std_dice_coeff = np.std(np.array(dice_coeffs))

        mean_roc_auc = np.nansum(roc_auc_coeffs) / len(roc_auc_coeffs)
        std_roc_auc = np.std(np.array(roc_auc_coeffs))

        precision_recall_auc = np.nansum(precision_recall_auc_coeffs) / len(precision_recall_auc_coeffs)
        std_roc_auc = np.std(np.array(precision_recall_auc_coeffs))

        mean_hausdorf_error = np.nansum(hausdorf_errors) / len(hausdorf_errors)
        std_hausdorf_error = np.std(np.array(hausdorf_errors))

        if writer is not None:
          writer.add_scalar("Metrics/ccr", mean_ccr, epoch)
          writer.add_scalar("Metrics/precision", mean_precision, epoch)
          writer.add_scalar("Metrics/recall", mean_recall, epoch)
          writer.add_scalar("Metrics/sensibility", mean_sensibility, epoch)
          writer.add_scalar("Metrics/specifity", mean_specifity, epoch)
          writer.add_scalar("Metrics/f1 score", mean_f1_score, epoch)
          writer.add_scalar("Metrics/jaccard idx", mean_jaccard_coef, epoch)
          writer.add_scalar("Metrics/dice coeff", mean_dice_coeff, epoch)
          writer.add_scalar("Metrics/roc-auc", mean_roc_auc, epoch)
          writer.add_scalar("Metrics/precision recall auc", precision_recall_auc, epoch)
          writer.add_scalar("Metrics/hausdorf error", mean_hausdorf_error, epoch)

        return SegmentationEvaluationMetrics(mean_ccr, mean_precision, mean_recall,
         mean_sensibility, mean_specifity, mean_f1_score, mean_jaccard_coef,
         mean_dice_coeff, mean_roc_auc, precision_recall_auc, mean_hausdorf_error)

In [15]:
sys.path.insert(0, '/content/drive/My Drive/Iñaki.Martínez/Code')

from dataset_handler import DataSet
from rdau_net import RDAU_NET
from discriminator import Discriminator


#writer = SummaryWriter('/content/drive/My Drive/Iñaki.Martínez/Code/runs/tensorboard_log')
torch.manual_seed(RANDOM_SEED)
train_data_transform, val_data_transform = load_img_transforms()

transforms_dict = {
    "train": train_data_transform,
    "val": train_data_transform,
    "test": train_data_transform
}

augmentation_dict = {
    "train": None,
    "val": None,
    "test": None
}

dataset = DataSet(TRAIN_DATA_DIR, TRAIN_DATA_ANNOTATIONS_FILE,TRAIN_DATA_ANNOTATIONS_FILE,TRAIN_DATA_ANNOTATIONS_FILE, transforms_dict, augmentation_dict, BATCH_SIZE, 2)

generator = RDAU_NET().to(DEVICE)
generator.apply(weights_init)

discriminator = Discriminator().to(DEVICE)
discriminator.apply(weights_init)

if PRETRAINED_WEIGHTS:
  generator.load_state_dict(torch.load(os.path.join(WEIHGHTS_PATH, "generator_weights")))
  discriminator.load_state_dict(torch.load(os.path.join(WEIHGHTS_PATH, "discriminator_weights")))

optimizerD = optim.RMSprop(discriminator.parameters(), lr=DISCRIMINATOR_LEARNING_RATE)
optimizerG = optim.RMSprop(generator.parameters(), lr=GENERATOR_LEARNING_RATE)

G_losses = []
D_losses = []

log_filename = "/content/drive/My Drive/Iñaki.Martínez/Code/execution_log.csv"
with open(log_filename, 'w') as file:
  file.write("epoch,generator_loss,critic_loss,ccr,precision,recall,sensibility,specifity,f1_score,jaccard_coef,dsc_coef,roc_auc,pr_auc,hausdorf_error\n")

for epoch in range(INITIAL_EPOCH, INITIAL_EPOCH+N_EPOCHS-1):

  t_init = time.time()


  for i, batched_sample in enumerate(dataset.trainset_loader):

    ############################
    #### DISCRIMINATOR ########
    #########################

    optimizerD.zero_grad()

    generator.eval()
    discriminator.train()

    images, masks = batched_sample["image"].to(DEVICE), batched_sample["mask"].to(DEVICE)

    images_with_gt = merge_images_with_masks(images, masks).to(DEVICE)

    segmentations = generator(images).detach()
    #segmentations = torch.autograd.Variable((segmentations > 0.5).float(), requires_grad=True)

    images_with_segmentations = merge_images_with_masks(images, segmentations).to(DEVICE)

    loss_D = -torch.mean(discriminator(images_with_gt)) + torch.mean(discriminator(images_with_segmentations))

    _gradient_penalty = gradient_penalty(discriminator, images_with_gt, images_with_segmentations, 10*2, DEVICE)
    loss_D += _gradient_penalty

    loss_D.backward()
    optimizerD.step()

    ############################
    #### GENERATOR ########
    #########################

    if epoch % N_CRITIC == 0:

      generator.train()
      discriminator.eval()

      optimizerG.zero_grad()

      segmentations = generator(images)
      generated_segmentations = merge_images_with_masks(images, segmentations).to(DEVICE)

      loss_G = -torch.mean(discriminator(generated_segmentations))

      loss_G.backward()
      optimizerG.step()

      #writer.add_scalar("Train loss/generator", loss_G.item(), epoch * len(dataset.trainset_loader) + i)
      #writer.add_scalar("Train loss/discriminator", loss_D.item(), epoch * len(dataset.trainset_loader) + i)

      if i % 5 == 0 or i % len(dataset.trainset_loader)+1 == len(dataset.trainset_loader):
        print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, N_EPOCHS, i % len(dataset.trainset_loader)+1, len(dataset.trainset_loader), loss_D.item(), loss_G.item())
        )

      # Encolamos los costes para calcular el medio de la epoch cuando esta termine
      G_losses.append(loss_G.item())
      D_losses.append(loss_D.item())

    else:
      if i % 5 == 0 or i % len(dataset.trainset_loader)+1 == len(dataset.trainset_loader):
        print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: -]"
        % (epoch, N_EPOCHS, i % len(dataset.trainset_loader)+1, len(dataset.trainset_loader), loss_D.item())
        )

      #writer.add_scalar("Train loss/discriminator", loss_D.item(), epoch * len(dataset.trainset_loader) + i)
      D_losses.append(loss_D.item())

    #break

  #if len(G_losses) > 0:
  #  writer.add_scalar("Per epoch train loss/generator", np.mean(np.array(G_losses)), epoch)
  #writer.add_scalar("Per epoch train loss/discriminator", np.mean(np.array(D_losses)), epoch)

  

  torch.save(generator.state_dict(), os.path.join(WEIGHTS_PATH, "generator_weights"))
  torch.save(discriminator.state_dict(), os.path.join(WEIGHTS_PATH, "discriminator_weights"))

  metrics = get_evaluation_metrics2(epoch, dataset.trainset_loader, generator, DEVICE, SAVE_SEGS=True,
                         COLOR=True, N_EPOCHS_SAVE=10, folder='/content/drive/My Drive/Iñaki.Martínez/Code/inference')
  
  if len(G_losses) > 0:
    generator_loss = np.mean(np.array(G_losses))
  else:
    generator_loss = np.nan

  with open(log_filename, 'a') as file:
    file.write(f"{epoch},{generator_loss},{np.mean(np.array(D_losses))},{metrics.CCR},{metrics.precision},{metrics.recall},{metrics.sensibility},{metrics.specifity},{metrics.f1_score},{metrics.jaccard},{metrics.dice},{metrics.roc_auc},{metrics.precision_recall_auc},{metrics.hausdorf_error}\n")

  G_losses = []
  D_losses = []
  
  t_end = time.time()
  print("Elapsed time for epoch {}: {:.2f}s".format(epoch, t_end-t_init))

  #break

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Train set length: 453
Val set length: 453
Test set length: 453
Mini-batches size:
	Train set: 8
	Val set: 8
	Test set: 8


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch 1/450] [Batch 1/8] [D loss: 13491.838867] [G loss: -]
[Epoch 1/450] [Batch 6/8] [D loss: 120.573013] [G loss: -]
[Epoch 1/450] [Batch 8/8] [D loss: 93.885155] [G loss: -]
Elapsed time for epoch 1: 16.35s
[Epoch 2/450] [Batch 1/8] [D loss: 78.388390] [G loss: -]
[Epoch 2/450] [Batch 6/8] [D loss: 41.509762] [G loss: -]
[Epoch 2/450] [Batch 8/8] [D loss: 61.103485] [G loss: -]
Elapsed time for epoch 2: 16.52s
[Epoch 3/450] [Batch 1/8] [D loss: 33.007534] [G loss: -]
[Epoch 3/450] [Batch 6/8] [D loss: 21.195820] [G loss: -]
[Epoch 3/450] [Batch 8/8] [D loss: 17.176867] [G loss: -]
Elapsed time for epoch 3: 16.78s
[Epoch 4/450] [Batch 1/8] [D loss: 23.244871] [G loss: -]
[Epoch 4/450] [Batch 6/8] [D loss: 11.453063] [G loss: -]
[Epoch 4/450] [Batch 8/8] [D loss: 19.081400] [G loss: -]
Elapsed time for epoch 4: 16.67s
[Epoch 5/450] [Batch 1/8] [D loss: 10.027678] [G loss: 1.546457]
[Epoch 5/450] [Batch 6/8] [D loss: 6.837816] [G loss: -2.674353]
[Epoch 5/450] [Batch 8/8] [D loss: 0.5

error: ignored